In [1]:
import tensorflow as tf
import numpy as np
import cv2
import glob
import os
import random
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
class preprocess:
    def __init__(self):
        self.sequences = []
    def imread(self,dir):
    # 문제 이미지 하나 => 블럭 이미지 시퀀스
        self.images = [cv2.imread(file,0) for file in glob.glob(dir)]
    # images = [cv2.imread(file,0) for file in glob.glob("./image/new/new/1/1/3/x/*.png")]
    # images = [cv2.imread(file,0) for file in glob.glob("./image/new/new/1/1/4/o/*.png")]
    # images = [cv2.imread(file,0) for file in glob.glob("./image/new/new/1/1/4/x/*.png")]
    # images = [cv2.imread(file,0) for file in glob.glob("./image/new/new/1/1/5/o/*.png")]
    # images = [cv2.imread(file,0) for file in glob.glob("./image/new/new/1/1/5/x/*.png")]
    # images = [cv2.imread(file,0) for file in glob.glob("./image/new/new/2/positive_1/*.png")]
    # images = [cv2.imread(file,0) for file in glob.glob("./image/new/new/2/negative/*.png")]
    def seqwrite(self,dir):
        for i,im in enumerate(self.images):
            self.sequences.append(self.get_bbs(im))
            print(i,"image to sequence")
        np.save(dir,self.sequences)
        self.sequences = []
    
    def get_bbs(self,image):
        image = image
        iterations = 3
        dilation_d = 1.1
        dilation_max = (100, 100)
        intersection_threshold = 0.05

        bbs = self.get_text_region_proposals(image)
        intermetiate_bbs = []
        bbs_l = self.get_bounding_boxs_of_passages(bbs, iterations, intersection_threshold, dilation_d, 
                                              dilation_max, intermetiate_bbs)
        seq_images = []
        for box in bbs_l:
            x,y,w,h = box
            x,y = max(x,0),max(y,0)
            im = np.copy(image[y:y+h,x:x+w])
            im = cv2.resize(im,(64,64))
            im = np.reshape(im,-1)
            seq_images.append(im)
        return seq_images
    
    def get_text_region_proposals(self,image, min_area=30, max_area=500, delta=1):
        '''
        Source: https://stackoverflow.com/questions/34398188/trying-to-plot-opencvs-mser-regions-using-matplotlib

        Parameters
        ----------

        min_area: int, default 30
            Suggested value: ~[5 - 50]

        max_area: int, default 500
            Suggested value: ~[500 - 800]

        delta: int, default 3
            Suggested value: ~[1-5]

        '''
        mser = cv2.MSER_create()
        mser.setMinArea(min_area)
        mser.setMaxArea(max_area)
        mser.setDelta(delta)
        boxes = []
        coordinates,_ = mser.detectRegions(image)
        bbs = []
        for coord in coordinates:
            bbox = cv2.boundingRect(coord)
            x, y, w, h = bbox
            if w < 10 or h < 10 or w/h > 5 or h/w > 5:
                continue
            bbs.append((x, y, w, h))
        return bbs
    def dilate_bbs(self, bbs, dilation_d, dilation_max):
        bbs_l = []
        for bb in bbs:
            (x, y, w, h) = bb
            (centre_x, centre_y) = (x + w/2, y + h/2)
            (x1, y1, x2, y2) = (x, y, x + w, y + h)

            dx = (centre_x - x1) * dilation_d
            dy = (centre_y - y1) * dilation_d

            if dilation_max:
                if dx > dilation_max[0]:
                    dx = dilation_max[0]
                if dy > dilation_max[1]:
                    dy = dilation_max[1]

            x1_l = centre_x - dx
            y1_l = centre_y - dy
            x2_l = centre_x + dx
            y2_l = centre_y + dy

            bb_l = (int(x1_l), int(y1_l), int(x2_l - x1_l), int(y2_l - y1_l))
            bbs_l.append(bb_l)
        return bbs_l

    def does_rect_overlap(self, bb1, bb2):
        (x1_1, y1_1, w_1, h_1) = bb1
        (x1_1, y1_1, x2_1, y2_1) = (x1_1, y1_1, x1_1 + w_1, y1_1 + h_1)

        (x1_2, y1_2, w_2, h_2) = bb2
        (x1_2, y1_2, x2_2, y2_2) = (x1_2, y1_2, x1_2 + w_2, y1_2 + h_2)

        h_overlaps = True
        v_overlaps = True
        if (x1_1 > x2_2) or (x2_1 < x1_2):
            h_overlaps = False
        if (y1_1 > y2_2) or (y2_1 < y1_2):
            v_overlaps = False
        return h_overlaps and v_overlaps

    def get_intersection_percentage(self, bb1, bb2):
        # How much of bb1 is intersecting with bb2
        if not self.does_rect_overlap(bb1, bb2):
            return 0
        # source: https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/
        (x1_1, y1_1, w_1, h_1) = bb1
        (x1_1, y1_1, x2_1, y2_1) = (x1_1, y1_1, x1_1 + w_1, y1_1 + h_1)

        (x1_2, y1_2, w_2, h_2) = bb2
        (x1_2, y1_2, x2_2, y2_2) = (x1_2, y1_2, x1_2 + w_2, y1_2 + h_2)

        x_a = max(x1_1, x1_2)
        y_a = max(y1_1, y1_2)
        x_b = min(x2_1, x2_2)
        y_b = min(y2_1, y2_2)

        inter_area = (x_b - x_a + 1) * (y_b - y_a + 1)

        bb1_area = (w_1 + 1) * (h_1 + 1)
        bb2_area = (w_2 + 1) * (h_2 + 1)

        intersection_percentage = inter_area / float(bb1_area)
        return intersection_percentage

    def get_intersection_percentage_matrix(self, bbs):
        n = len(bbs)
        ip = np.zeros(shape=(n, n))
        for i in range(n):
            for j in range(n):
                ip[i, j] = self.get_intersection_percentage(bbs[i], bbs[j])
        return ip 

    def make_new_bb_from_intersect_bbs(self, bbs):
        max_x = np.max([bb[0] + bb[2] for bb in bbs])
        min_x = np.min([bb[0] for bb in bbs])
        max_y = np.max([bb[1] + bb[3] for bb in bbs])
        min_y = np.min([bb[1] for bb in bbs])
        return (min_x, min_y, max_x - min_x, max_y - min_y)

    def reduce_bbs_by_inetersection_percentage(self, bbs, in_matrix_thresholded):
        new_bbs = []
        for i in range(in_matrix_thresholded.shape[0]):
            in_i = in_matrix_thresholded[i, :]
            intersections_indexes = np.where(in_i)[0]
            if len(intersections_indexes) == 0:
                new_bbs.append(bbs[i])
            else:
                intersection_bbs = [bbs[i]]
                for intersections_index in intersections_indexes:
                    intersection_bbs.append(bbs[intersections_index])
                new_bb = self.make_new_bb_from_intersect_bbs(intersection_bbs)
                if new_bb not in new_bbs:
                    new_bbs.append(new_bb)
        return new_bbs

    '''
    def contains(bb1, bb2):
        # bb2 is in bb1
        # source: https://stackoverflow.com/questions/21275714/check-rectangle-inside-rectangle-in-python
        in_x = bb1[0] < bb2[0] < bb2[0] + bb2[2] < bb1[0] + bb1[2]
        in_y = bb1[1] < bb2[1] < bb2[1] + bb2[3] < bb1[1] + bb1[3]
        return in_x and in_y

    def remove_bbs_in_another_bb(bbs):
        new_bbs = []
        for i in range(len(bbs)):
            should_output_i = True
            bb_i = bbs[i]
            for j in range(len(bbs)):
                bb_j = bbs[j]
                if contains(bb_j, bb_i):
                    should_output_i = False
                    break
            if should_output_i:
                new_bbs.append(bb_i)
        return new_bbs
    '''

    def reduce_intersecting_bbs(self, bbs, intersection_threshold):
        new_bbs = bbs
        while True:
            old_bbs_no = len(bbs)
            in_matrix = self.get_intersection_percentage_matrix(bbs)
            in_matrix_thresholded = in_matrix > intersection_threshold
            bbs = self.reduce_bbs_by_inetersection_percentage(bbs, in_matrix_thresholded)
            if old_bbs_no == len(bbs):
                break
        return bbs

    def get_bounding_boxs_of_passages(self, bbs, iterations, ip_threshold, dilation_d, dilation_max=None,
                                      intermetiate_bbs=None):
        '''    
        Parameters
        ----------
        iterations: int

        ios_threshold: 

        dilation_d: float
            percentage to increase the bounding box

        '''
        for i in range(iterations):
            bbs = self.dilate_bbs(bbs, dilation_d, dilation_max)
            bbs = self.reduce_intersecting_bbs(bbs, ip_threshold)
            if intermetiate_bbs is not None:
                intermetiate_bbs.append(bbs)
#             print("Running {}/{}, new #bbs {}".format(i, iterations, len(bbs)))
        return bbs


In [3]:
class readdata:
    def __init__(self):
        self.pairs = []
        self.index = 0
        images_o = np.load("./data/seq/seq_3_o.npy")
        images_x = np.load("./data/seq/seq_3_x.npy")
        for i in range(len(images_o)-1):
            for j in range(i+1,len(images_o)):
                for k in images_x:
                    self.pairs.append([images_o[i],images_o[j],k])
        random.shuffle(self.pairs)
    def next_batch(self,batch_size):
        x = []
        x_p = []
        x_n = []
        for i in range(batch_size):
            if self.index > len(self.pairs)-1:
                self.index = 0
            x.append(self.pairs[self.index][0])
            x_p.append(self.pairs[self.index][1])
            x_n.append(self.pairs[self.index][2])
            self.index = self.index + 1
        return x, x_p, x_n


In [4]:
batch = readdata()

In [268]:
x,p,n = batch.next_batch(1)

In [275]:
for i in range(10000):
    x,x_p,x_n = batch.next_batch(1)
    if len(x_n[0]) >= 45:
        print(i,len(x_n[0]))

12 47
43 56
51 48
84 47
115 56
123 48
156 47
187 56
195 48
228 47
259 56
267 48
300 47
331 56
339 48
372 47
403 56
411 48
444 47
475 56
483 48
516 47
547 56
555 48
588 47
619 56
627 48
660 47
691 56
699 48
732 47
763 56
771 48
804 47
835 56
843 48
876 47
907 56
915 48
948 47
979 56
987 48
1020 47
1051 56
1059 48
1092 47
1123 56
1131 48
1164 47
1195 56
1203 48
1236 47
1267 56
1275 48
1308 47
1339 56
1347 48
1380 47
1411 56
1419 48
1452 47
1483 56
1491 48
1524 47
1555 56
1563 48
1596 47
1627 56
1635 48
1668 47
1699 56
1707 48
1740 47
1771 56
1779 48
1812 47
1843 56
1851 48
1884 47
1915 56
1923 48
1956 47
1987 56
1995 48
2028 47
2059 56
2067 48
2100 47
2131 56
2139 48
2172 47
2203 56
2211 48
2244 47
2275 56
2283 48
2316 47
2347 56
2355 48
2388 47
2419 56
2427 48
2460 47
2491 56
2499 48
2532 47
2563 56
2571 48
2604 47
2635 56
2643 48
2676 47
2707 56
2715 48
2748 47
2779 56
2787 48
2820 47
2851 56
2859 48
2892 47
2923 56
2931 48
2964 47
2995 56
3003 48
3036 47
3067 56
3075 48
3108 47
3139 5

In [5]:
class lstm:
    def __init__(self, sequence_length, input_size, hidden_units, batch_size, alpha):
        self.input_x = tf.placeholder(tf.float32, [None, sequence_length, input_size], name="input_x")
        self.input_x_p = tf.placeholder(tf.float32, [None, sequence_length, input_size], name="input_x_p")
        self.input_x_n = tf.placeholder(tf.float32, [None, sequence_length, input_size], name="input_x_n")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        with tf.variable_scope("output"):
            self.out1=self.birnn(self.input_x, self.dropout_keep_prob, "lstm", sequence_length, hidden_units) #[batch][cell]
            self.out2=self.birnn(self.input_x_p, self.dropout_keep_prob, "lstm", sequence_length, hidden_units)
            self.out3=self.birnn(self.input_x_n, self.dropout_keep_prob, "lstm", sequence_length, hidden_units)
            self.distance_p = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(self.out1, self.out2)),axis=1)) #[batch]
            self.distance_n = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(self.out1, self.out3)),axis=1))

        with tf.variable_scope("loss"):
            self.loss = tf.reduce_sum(tf.maximum(tf.square(self.distance_p)-tf.square(self.distance_n)+alpha,0),axis=0)
            
    def birnn(self, x, dropout, scope, sequence_length, hidden_units):
        n_hidden=hidden_units
        n_layers=3
        x = tf.unstack(tf.transpose(x, perm=[1, 0, 2]))
        with tf.variable_scope("fw"+scope,reuse=tf.AUTO_REUSE):
            stacked_rnn_fw = []
            for _ in range(n_layers):
                fw_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
                lstm_fw_cell = tf.nn.rnn_cell.DropoutWrapper(fw_cell,output_keep_prob=dropout)
                stacked_rnn_fw.append(lstm_fw_cell)
            lstm_fw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_fw, state_is_tuple=True)
        with tf.variable_scope("bw"+scope,reuse=tf.AUTO_REUSE):
            stacked_rnn_bw = []
            for _ in range(n_layers):
                bw_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
                lstm_bw_cell = tf.nn.rnn_cell.DropoutWrapper(bw_cell,output_keep_prob=dropout)
                stacked_rnn_bw.append(lstm_bw_cell)
            lstm_bw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_bw, state_is_tuple=True)
        with tf.variable_scope("bw"+scope,reuse=tf.AUTO_REUSE):
            outputs, _, _ = tf.nn.static_bidirectional_rnn(lstm_fw_cell_m, lstm_bw_cell_m, x, dtype=tf.float32) # [time][batch][cell]
        return outputs[-1] #[batch][cell]


In [6]:
input_size = 64*64
sequence_length = 60
hidden_units = 128
batch_size = 4
dropout_keep_prob = 0.8
epochs = 5
alpha = 2.0

In [7]:
with tf.Graph().as_default():
    sess = tf.Session()
    print("started session")
    with sess.as_default():
        siam = lstm(sequence_length,input_size, hidden_units, batch_size, alpha)
        print("initialized siameseModel object")
    train_op = tf.train.AdamOptimizer(0.05).minimize(siam.loss)
    print("defined training_ops")

    #     

    checkpoint_dir = "./ckpt"
    checkpoint_prefix = os.path.join(checkpoint_dir, "model")
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
        
    sess.run(tf.global_variables_initializer())
    print("init all variables")
    graph_def = tf.get_default_graph().as_graph_def()
    graphpb = str(graph_def)
    with open(os.path.join(checkpoint_dir, "graph.pb"), 'w') as f:
        f.write(graphpb)
    def train_step(x_batch, x_p_batch, x_n_batch, iter):
        x_padded = []
        x_p_padded = [] #[batch][time][input]
        x_n_padded = []
        for x in x_batch:
            if len(x) < sequence_length:
                x_tem = np.append(x,np.zeros([sequence_length-len(x),4096]),axis=0)
            else:
                x_tem = x
            x_padded.append(x_tem)
        for x in x_p_batch:
            if len(x) < sequence_length:
                x_tem = np.append(x,np.zeros([sequence_length-len(x),4096]),axis=0)
            else:
                x_tem = x
            x_p_padded.append(x_tem)
        for x in x_n_batch:
            if len(x) < sequence_length:
                x_tem = np.append(x,np.zeros([sequence_length-len(x),4096]),axis=0)
            else:
                x_tem = x
            x_n_padded.append(x_tem)
        feed_dict = {
            siam.input_x: x_padded,
            siam.input_x_p: x_p_padded,
            siam.input_x_n: x_n_padded,
            siam.dropout_keep_prob: dropout_keep_prob
        }
        _, loss, dist_p, dist_n = sess.run([train_op, siam.loss, siam.distance_p, siam.distance_n],feed_dict=feed_dict)
        print("TRAIN step {}, loss {:g}".format(iter,loss))
        print(dist_p[0:10])
        print(dist_n[0:10])

started session
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


KeyboardInterrupt: 

In [ ]:
n_batchs = (int)(len(batch.pairs) / batch_size)
for i in range(n_batchs * epochs):
    x, x_p, x_n = batch.next_batch(batch_size)
    train_step(x, x_p, x_n, i)